**ASSIGNMENT 4**

1. Group Name: Group 72 \\
   Member Names: Angus Leung \\
   Member Student Numbers: 300110509 \\
   Report Title: Logistic Regression and MLP Classifiers for NLP Sentiment Analysis \\

**Derived Datasets**

This notebook is a starting point for Assignment 4. In this assignment, you will perform a classification empirical study. This notebook will help you to create derived datasets in Section 2 of the assignment.

In [1]:
#let's start by installing spaCy
!pip install spacy
# Install Sci Kit
!pip install scikit-learn

In [2]:
import spacy
import pandas as pd
import numpy as np

You have been given a list of datasets in the assignment description. Choose one of the datasets and provide the link below and read the dataset using pandas. You should provide a link to your own Github repository even if you are using a reduced version of a dataset from your TA's repository.

add description of the dataset and your justification of the choices made to obtain the derived datasets

In [3]:
#Load the dataset you chose.
# Make sure the Notebook can load your dataset, just like previous assignments.

# url = 'https://raw.githubusercontent.com/baharin/CSI4106-Assignment4-Datasets/main/reduced_file_cnnnews.csv'
# url = 'https://raw.githubusercontent.com/baharin/CSI4106-Assignment4-Datasets/main/reduced_drugsComTest_raw_fiveclasses.csv'
# url = 'https://raw.githubusercontent.com/baharin/CSI4106-Assignment4-Datasets/main/reduced_file_AirPassengerReviews.csv'

#provide the link to the raw version of dataset. You *need* to provide a link to *your own* github repository. DO NOT use the link that is provided as an example.
url = "https://raw.githubusercontent.com/angusleung100/CSI4106-A4/main/reduced_file_AirPassengerReviews.csv"




In [4]:
print(url)
data = pd.read_csv(url)

https://raw.githubusercontent.com/angusleung100/CSI4106-A4/main/reduced_file_AirPassengerReviews.csv


In [5]:
data.head()

,customer_review,NPS Score
0,London to Izmir via Istanbul. First time I'd ...,Passive
1,Istanbul to Bucharest. We make our check in i...,Detractor
2,Rome to Prishtina via Istanbul. I flew with t...,Detractor
3,Flew on Turkish Airlines IAD-IST-KHI and retu...,Promoter
4,Mumbai to Dublin via Istanbul. Never book Tur...,Detractor


This is where you create the NLP pipeline. load() will download the correct model (English).

In [6]:
nlp = spacy.load("en_core_web_sm")

Applying the pipeline to every sentences creates a Document where every word is a Token object.

Doc: https://spacy.io/api/doc

Token: https://spacy.io/api/token

In [7]:
#Apply nlp pipeline to the column that has your sentences.
data['tokenized'] = data['customer_review'].apply(nlp)

In [8]:
data.head()

,customer_review,NPS Score,tokenized
0,London to Izmir via Istanbul. First time I'd ...,Passive,"( , London, to, Izmir, via, Istanbul, ., First..."
1,Istanbul to Bucharest. We make our check in i...,Detractor,"( , Istanbul, to, Bucharest, ., We, make, our,..."
2,Rome to Prishtina via Istanbul. I flew with t...,Detractor,"( , Rome, to, Prishtina, via, Istanbul, ., I, ..."
3,Flew on Turkish Airlines IAD-IST-KHI and retu...,Promoter,"( , Flew, on, Turkish, Airlines, IAD, -, IST, ..."
4,Mumbai to Dublin via Istanbul. Never book Tur...,Detractor,"( , Mumbai, to, Dublin, via, Istanbul, ., Neve..."


A Token object has many attributes such as part-of-speech (pos_), lemma (lemma_), etc. Take a look at the documentation to see all attributes.

The following function is an example on how you can fetch a specific pos tagging from a sentence. We return the lemmatization because we only want the infinitive word.

In [9]:
#create empty dataframes that will store your derived datasets

derived_dataset1 = pd.DataFrame(columns = ['Class', 'pos'])
derived_dataset2 = pd.DataFrame(columns = ['Class', 'pos-np'])

In [10]:
def get_pos(sentence, wanted_pos): #wanted_pos refers to the desired pos tagging
    verbs = []
    for token in sentence:
        if token.pos_ in wanted_pos:
            verbs.append(token.lemma_) # lemma returns a number. lemma_ return a string
    return ' '.join(verbs) # return value is as a string and not a list for countVectorizer

In [11]:
#As an example, we use the above function to fetch all the verbs. We store this information in our first derived dataset
derived_dataset1['pos'] = data['tokenized'].apply(lambda sent : get_pos(sent, ['VERB', 'ADJ']))

In [12]:
derived_dataset1.head()

,Class,pos
0,NaN,first fly find good nice great delay miss Most...
1,NaN,make take go let ™ say overbooke have wait bri...
2,NaN,fly several past say get bad bad fly have norm...
3,NaN,return maintain fly leave excellent inflight e...
4,NaN,turkish travel get have other ask stay care ma...


In [13]:
#Change this line to fetch your desired pos taggings for the second derived dataset
derived_dataset2['pos-np'] = data['tokenized'].apply(lambda sent : get_pos(sent, ['ADJ']))

derived_dataset2.head()

,Class,pos-np
0,NaN,first good nice great Most contradictory littl...
1,NaN,first last
2,NaN,several past bad bad normal most useless few w...
3,NaN,excellent inflight extensive easy excellent in...
4,NaN,turkish other more


In [14]:
#For Derived Dataset 2, you also need to include Named Entities
#Below is just an example of obtaining such entities on a specific sentence, but you would do NER
#on the dataset of your choice.
#You can choose the types of entities (dates, organization, people) that you want,
#and then in your derived dataset, just make sure you include these entities separated by spaces (as shown for verbs)
#in a previous cell.

# Note: Modified example to become a function and filter out what we need with wanted name entities
def get_name_entities(sentence, wanted_name_entities):

  doc = nlp(sentence)

  entities = []

  for ent in doc.ents:

      if ent.label_ in wanted_name_entities:
        entities.append(ent.text)
  return ' '.join(entities)


In [15]:
# Append name entities to derived dataset 2 with adj in pos-np column
derived_dataset2['pos-np'] += ' ' + data['tokenized'].apply(lambda sent : get_name_entities(sent, ['DATE', 'GPE', 'MONEY', 'ORG']))

In [16]:
derived_dataset2.head()

,Class,pos-np
0,NaN,first good nice great Most contradictory littl...
1,NaN,first last Istanbul
2,NaN,several past bad bad normal most useless few w...
3,NaN,excellent inflight extensive easy excellent in...
4,NaN,turkish other more Mumbai Dublin Istanbul Dubl...


Now that you have your derived datasets, you can move to perform your classificaton task.

**Perform Classification Task**

In [27]:
# Import required
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn import metrics

from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split


from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score
from numpy import mean
from numpy import std

from sklearn.neural_network import MLPClassifier

## Encode text as input features with associated values

In [18]:
# Encode the score values
def encodeScores(targetDataset):
  targetScore = [] # NPS Score to int value

  for i, row in targetDataset.iterrows():
    if row['NPS Score'] == "Detractor":
      targetScore.append(0)
    elif row['NPS Score'] == "Promoter":
      targetScore.append(1)
    elif row['NPS Score'] == "Passive":
      targetScore.append(2)
    else:
      targetScore.append(3)

  return targetScore


In [19]:
# Data Sets Used

# Original Data
originalData = data.copy()

XOriginal = originalData['tokenized'].astype(str)
YOriginal = encodeScores(originalData)

XTrainZero, XTestZero, YTrainZero, YTestZero = train_test_split(XOriginal, YOriginal, test_size=0.2, random_state=42)

# Derivative 1 Data
derivedDataOne = derived_dataset1.copy()
XOne = derivedDataOne['pos'].astype(str)
YOne = YOriginal

XTrainOne, XTestOne, YTrainOne, YTestOne = train_test_split(XOne, YOne, test_size=0.2, random_state=42)

# Derivative 2 Data
derivedDataTwo = derived_dataset2.copy()
XTwo = derivedDataTwo['pos-np'].astype(str)
YTwo = YOriginal

XTrainTwo, XTestTwo, YTrainTwo, YTestTwo = train_test_split(XTwo, YTwo, test_size=0.2, random_state=42)

## Logistic Regression with default parameters

In [52]:
# Pipelines Default Parameters Train on Logistic Regreg

LogRegDataOrig = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()), ('clf', LogisticRegression()),])
LogRegDerivOne = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()), ('clf', LogisticRegression()),])
LogRegDerivTwo = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()), ('clf', LogisticRegression()),])

#LogRegDataOrig.fit(XTrainZero, YTrainZero)
#LogRegDerivOne.fit(XTrainOne, YTrainOne)
#LogRegDerivTwo.fit(XTrainTwo, YTrainTwo)

In [51]:
# Check accuracy of logistic regression model pipeline
#predictLogRegOrig = LogRegDataOrig.predict(XTestZero)
#print(np.mean(YTestZero == predictLogRegOrig))

#predictLogRegOne = LogRegDataOrig.predict(XTestOne)
#print(np.mean(YTestZero == predictLogRegOne))

#predictLogRegTwo = LogRegDataOrig.predict(XTestTwo)
#print(np.mean(YTestZero == predictLogRegTwo))

## MLP Classifier

In [50]:
# Pipelines Default Parameters Train on MLP

# Default parameters (Will be changed later for testing):
# learning_rate = constant
# max_iter = 200

MLPOrig = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()), ('clf', MLPClassifier()),])
#MLPOrig = MLPOrig.set_params()
#MLPOrig = MLPOrig.fit(XTrainZero, YTrainZero)
#print(MLPOrig.score(XTestZero, YTestZero))

MLPOne = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()), ('clf', MLPClassifier()),])
#MLPOne = MLPOne.set_params()
#MLPOne = MLPOne.fit(XTrainOne, YTrainOne)
#print(MLPOne.score(XTestOne, YTestOne))

MLPTwo = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()), ('clf', MLPClassifier()),])
#MLPTwo = MLPTwo.set_params()
#MLPTwo = MLPTwo.fit(XTrainTwo, YTrainTwo)
#print(MLPTwo.score(XTestTwo, YTestTwo))

## Testing and Evaluating with scoring (Default params)

In [28]:
# Score output
def printScores(scores):
  print("Accuracy: "+str(mean(scores['test_accuracy']))+" | SD: "+str(std(scores['test_accuracy'])))
  print("Precision (Micro): "+str(mean(scores['test_precision_micro']))+" | SD: "+str(std(scores['test_precision_micro'])))
  print("Precision (Macro): "+str(mean(scores['test_precision_macro']))+" | SD: "+str(std(scores['test_precision_macro'])))
  print("Recall (Micro): "+str(mean(scores['test_recall_micro']))+" | SD: "+str(std(scores['test_recall_micro'])))
  print("Recall (Macro): "+str(mean(scores['test_recall_macro']))+" | SD: "+str(std(scores['test_recall_macro'])))

In [31]:
# Logistic Regression

# 4 Fold
crossValidate = KFold(n_splits=4)

# scoring settings
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision_micro': make_scorer(precision_score, average='micro'),
           'precision_macro': make_scorer(precision_score, average='macro'),
           'recall_micro': make_scorer(recall_score, average='micro'),
           'recall_macro': make_scorer(recall_score, average='macro')}

print("Logistic Regression Original Data")
LogRegDataOrigScores = cross_validate(LogRegDataOrig, XOriginal, YOriginal, scoring=scoring, cv=crossValidate, n_jobs=-1, error_score="raise")
printScores(LogRegDataOrigScores)

print("Logistic Regression Derived 1 Data")
LogRegDataOneScores = cross_validate(LogRegDerivOne, XOne, YOne, scoring=scoring, cv=crossValidate, n_jobs=-1, error_score="raise")
printScores(LogRegDataOneScores)

print("Logistic Regression Derived 2 Data")
LogRegDataTwoScores = cross_validate(LogRegDerivTwo, XTwo, YTwo, scoring=scoring, cv=crossValidate, n_jobs=-1, error_score="raise")
printScores(LogRegDataTwoScores)



Logistic Regression Original Data
Accuracy: 0.7871979985605329 | SD: 0.04974479263055521
Precision (Micro): 0.7871979985605329 | SD: 0.04974479263055521
Precision (Macro): 0.6979936436803671 | SD: 0.012913034455655725
Recall (Micro): 0.7871979985605329 | SD: 0.04974479263055521
Recall (Macro): 0.6704528084311742 | SD: 0.01926447895149705
Logistic Regression Derived 1 Data
Accuracy: 0.779298957422442 | SD: 0.050512791524743327
Precision (Micro): 0.779298957422442 | SD: 0.050512791524743327
Precision (Macro): 0.6791830098882504 | SD: 0.021009586000992772
Recall (Micro): 0.779298957422442 | SD: 0.050512791524743327
Recall (Macro): 0.6573145033102018 | SD: 0.013391767119004423
Logistic Regression Derived 2 Data
Accuracy: 0.7505864239741724 | SD: 0.06124670347273329
Precision (Micro): 0.7505864239741724 | SD: 0.06124670347273329
Precision (Macro): 0.6523193245532445 | SD: 0.013339166325653807
Recall (Micro): 0.7505864239741724 | SD: 0.06124670347273329
Recall (Macro): 0.6297385369368745 | S

In [32]:
# MLP

# 4 Fold
crossValidate = KFold(n_splits=4)

# scoring settings
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision_micro': make_scorer(precision_score, average='micro'),
           'precision_macro': make_scorer(precision_score, average='macro'),
           'recall_micro': make_scorer(recall_score, average='micro'),
           'recall_macro': make_scorer(recall_score, average='macro')}

print("MLP Original Data")
MLPDataOrigScores = cross_validate(MLPOrig, XOriginal, YOriginal, scoring=scoring, cv=crossValidate, n_jobs=-1, error_score="raise")
printScores(MLPDataOrigScores)

print("MLP Derived 1 Data")
MLPDataOneScores = cross_validate(MLPOne, XOne, YOne, scoring=scoring, cv=crossValidate, n_jobs=-1, error_score="raise")
printScores(MLPDataOneScores)

print("MLP Derived 2 Data")
MLPDataTwoScores = cross_validate(MLPTwo, XTwo, YTwo, scoring=scoring, cv=crossValidate, n_jobs=-1, error_score="raise")
printScores(MLPDataTwoScores)

MLP Original Data
Accuracy: 0.7526297558017293 | SD: 0.05728441566919431
Precision (Micro): 0.7526297558017293 | SD: 0.05728441566919431
Precision (Macro): 0.6574139066691673 | SD: 0.011893024194706897
Recall (Micro): 0.7526297558017293 | SD: 0.05728441566919431
Recall (Macro): 0.6596520012090223 | SD: 0.0077860501918799105
MLP Derived 1 Data
Accuracy: 0.7045830173274815 | SD: 0.04228638563238359
Precision (Micro): 0.7045830173274815 | SD: 0.04228638563238359
Precision (Macro): 0.6072071696573753 | SD: 0.02209754869433878
Recall (Micro): 0.7045830173274815 | SD: 0.04228638563238359
Recall (Macro): 0.6195157883091523 | SD: 0.0086934773882508
MLP Derived 2 Data
Accuracy: 0.6582164975085615 | SD: 0.06432506626970061
Precision (Micro): 0.6582164975085615 | SD: 0.06432506626970061
Precision (Macro): 0.5739112344084412 | SD: 0.01598142091544596
Recall (Micro): 0.6582164975085615 | SD: 0.06432506626970061
Recall (Macro): 0.5807931570906334 | SD: 0.00941861958807479


## Discussion of Results for Model Train/Test/Eval

### Logistic Regression Default Params
With reduced datasets for faster computation, we seem to have our best accuracy when the text is not tokenized heavily. The precision on both micro and macro are significantly higher on the first set vs second set.

Below is a table of the results and it shows that the more we try to filter tokens, the more difficult it gets for a more accurate result with this reduced dataset somehow.

```
Logistic Regression Original Data
Accuracy: 0.7871979985605329 | SD: 0.04974479263055521
Precision (Micro): 0.7871979985605329 | SD: 0.04974479263055521
Precision (Macro): 0.6979936436803671 | SD: 0.012913034455655725
Recall (Micro): 0.7871979985605329 | SD: 0.04974479263055521
Recall (Macro): 0.6704528084311742 | SD: 0.01926447895149705
Logistic Regression Derived 1 Data
Accuracy: 0.779298957422442 | SD: 0.050512791524743327
Precision (Micro): 0.779298957422442 | SD: 0.050512791524743327
Precision (Macro): 0.6791830098882504 | SD: 0.021009586000992772
Recall (Micro): 0.779298957422442 | SD: 0.050512791524743327
Recall (Macro): 0.6573145033102018 | SD: 0.013391767119004423
Logistic Regression Derived 2 Data
Accuracy: 0.7505864239741724 | SD: 0.06124670347273329
Precision (Micro): 0.7505864239741724 | SD: 0.06124670347273329
Precision (Macro): 0.6523193245532445 | SD: 0.013339166325653807
Recall (Micro): 0.7505864239741724 | SD: 0.06124670347273329
Recall (Macro): 0.6297385369368745 | SD: 0.0159129705757223
```

Micro results were about the same for Precision and recall. Based on such a reduced dataset, it would be hard to differentiate a micro and a macro calculation as both would be working with smaller numbers of rows. The classes in general are scaled down in terms of testing and training data, which would result in much similar micro and macro.

### MLP Default Params
The default MLP parameters takes a very long time (>8 min per dataset) to compute through the 3 datasets. This is most likely due to the fact that instead of a standard math model, Scikit would need to pass the training and testing data through a Neural Network model.

```
MLP Original Data
Accuracy: 0.7526297558017293 | SD: 0.05728441566919431
Precision (Micro): 0.7526297558017293 | SD: 0.05728441566919431
Precision (Macro): 0.6574139066691673 | SD: 0.011893024194706897
Recall (Micro): 0.7526297558017293 | SD: 0.05728441566919431
Recall (Macro): 0.6596520012090223 | SD: 0.0077860501918799105
MLP Derived 1 Data
Accuracy: 0.7045830173274815 | SD: 0.04228638563238359
Precision (Micro): 0.7045830173274815 | SD: 0.04228638563238359
Precision (Macro): 0.6072071696573753 | SD: 0.02209754869433878
Recall (Micro): 0.7045830173274815 | SD: 0.04228638563238359
Recall (Macro): 0.6195157883091523 | SD: 0.0086934773882508
MLP Derived 2 Data
Accuracy: 0.6582164975085615 | SD: 0.06432506626970061
Precision (Micro): 0.6582164975085615 | SD: 0.06432506626970061
Precision (Macro): 0.5739112344084412 | SD: 0.01598142091544596
Recall (Micro): 0.6582164975085615 | SD: 0.06432506626970061
Recall (Macro): 0.5807931570906334 | SD: 0.00941861958807479
```

Based on the results as shown above, an unfiltered tokenized review yielded a higher accuracy than the rest of the MLP models on derive 1 and derive 2 datasets.

Again, precision and recall on a micro level are practically identical, which I again assume is due to the scaled down datasets, making it much harder to deviate from that precision and recall with alternative testing data.

## Logistic Regression vs MLP
The Logistic Regression model seemed to be more accurate. On a micro and macro level, it still beats the MLP classifier. I believe it does this due to the reduced datasets. Since MLP is a Neural Network classifier, it requires a larger dataset to properly train the neural network to identify. Continuously putting input data into the neural network would yield much better results, but due to the lack of computing power that Google Colab has for our class assignments, we have to settle with reduced datasets and thus, lower scores.

Logistic Regression is a primarily math model and does not exactly "learn" from more inputs. It just grabs as many inputs and plots the best regression. This I believe is much better for the reduced dataset and is the reason why the accuracy is a bit better. Okay, both would do much better with larger datasets but linear regression would use what it has to plot while neural network would built upon itself. That is what is the difference on accuracy score raising.


# Modify MLP Model parameters

In [44]:
# Pipelines Custom Parameters Train on MLP

# New Custom Parameters:
# learning_rate = adaptive

MLPOrig = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()), ('clf', MLPClassifier()),])
MLPOrig = MLPOrig.set_params(clf__learning_rate='adaptive')
#MLPOrig = MLPOrig.fit(XTrainZero, YTrainZero)
#print(MLPOrig.score(XTestZero, YTestZero))

MLPOne = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()), ('clf', MLPClassifier()),])
MLPOne = MLPOne.set_params(clf__learning_rate='adaptive')
#MLPOne = MLPOne.fit(XTrainOne, YTrainOne)
#print(MLPOne.score(XTestOne, YTestOne))

MLPTwo = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()), ('clf', MLPClassifier()),])
MLPTwo = MLPTwo.set_params(clf__learning_rate='adaptive')
#MLPTwo = MLPTwo.fit(XTrainTwo, YTrainTwo)
#print(MLPTwo.score(XTestTwo, YTestTwo))

In [46]:
# Run changed parameter MLP Model

print("MLP Original Data (Custom Parameter)")
MLPDataOrigScores = cross_validate(MLPOrig, XOriginal, YOriginal, scoring=scoring, cv=crossValidate, n_jobs=-1, error_score="raise")
printScores(MLPDataOrigScores)

print("MLP Derived 1 Data (Custom Parameter)")
MLPDataOneScores = cross_validate(MLPOne, XOne, YOne, scoring=scoring, cv=crossValidate, n_jobs=-1, error_score="raise")
printScores(MLPDataOneScores)

print("MLP Derived 2 Data (Custom Parameter)")
MLPDataTwoScores = cross_validate(MLPTwo, XTwo, YTwo, scoring=scoring, cv=crossValidate, n_jobs=-1, error_score="raise")
printScores(MLPDataTwoScores)

MLP Original Data (Custom Parameter)
Accuracy: 0.7530013303328318 | SD: 0.05515907375272302
Precision (Micro): 0.7530013303328318 | SD: 0.05515907375272302
Precision (Macro): 0.6580091451164207 | SD: 0.01369859669343959
Recall (Micro): 0.7530013303328318 | SD: 0.05515907375272302
Recall (Macro): 0.6608877356508466 | SD: 0.008622120813757373
MLP Derived 1 Data (Custom Parameter)
Accuracy: 0.7036537874423765 | SD: 0.04299811615073797
Precision (Micro): 0.7036537874423765 | SD: 0.04299811615073797
Precision (Macro): 0.6056796924886272 | SD: 0.021692283209374524
Recall (Micro): 0.7036537874423765 | SD: 0.04299811615073797
Recall (Macro): 0.6175541459084531 | SD: 0.008881357419671118
MLP Derived 2 Data (Custom Parameter)
Accuracy: 0.6537560144720319 | SD: 0.0658150325771587
Precision (Micro): 0.6537560144720319 | SD: 0.0658150325771587
Precision (Macro): 0.570238699382446 | SD: 0.014253218148431383
Recall (Micro): 0.6537560144720319 | SD: 0.0658150325771587
Recall (Macro): 0.576781362352444

In [47]:
# Pipelines Custom Parameters Train on MLP

# New Custom Parameters:
# max_iter = 100

MLPOrig = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()), ('clf', MLPClassifier()),])
MLPOrig = MLPOrig.set_params(clf__max_iter=100)
#MLPOrig = MLPOrig.fit(XTrainZero, YTrainZero)
#print(MLPOrig.score(XTestZero, YTestZero))

MLPOne = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()), ('clf', MLPClassifier()),])
MLPOne = MLPOne.set_params(clf__max_iter=100)
#MLPOne = MLPOne.fit(XTrainOne, YTrainOne)
#print(MLPOne.score(XTestOne, YTestOne))

MLPTwo = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()), ('clf', MLPClassifier()),])
MLPTwo = MLPTwo.set_params(clf__max_iter=100)
#MLPTwo = MLPTwo.fit(XTrainTwo, YTrainTwo)
#print(MLPTwo.score(XTestTwo, YTestTwo))

In [49]:
# Run changed parameter MLP Model

print("MLP Original Data (Custom Parameter)")
MLPDataOrigScores = cross_validate(MLPOrig, XOriginal, YOriginal, scoring=scoring, cv=crossValidate, n_jobs=-1, error_score="raise")
printScores(MLPDataOrigScores)

print("MLP Derived 1 Data (Custom Parameter)")
MLPDataOneScores = cross_validate(MLPOne, XOne, YOne, scoring=scoring, cv=crossValidate, n_jobs=-1, error_score="raise")
printScores(MLPDataOneScores)

print("MLP Derived 2 Data (Custom Parameter)")
MLPDataTwoScores = cross_validate(MLPTwo, XTwo, YTwo, scoring=scoring, cv=crossValidate, n_jobs=-1, error_score="raise")
printScores(MLPDataTwoScores)

MLP Original Data (Custom Parameter)
Accuracy: 0.7517003877719894 | SD: 0.05479951527999845
Precision (Micro): 0.7517003877719894 | SD: 0.05479951527999845
Precision (Macro): 0.6563141348097064 | SD: 0.012381907763841111
Recall (Micro): 0.7517003877719894 | SD: 0.05479951527999845
Recall (Macro): 0.6589889671637601 | SD: 0.007750414878090436
MLP Derived 1 Data (Custom Parameter)
Accuracy: 0.7023524995199473 | SD: 0.04362806716890574
Precision (Micro): 0.7023524995199473 | SD: 0.04362806716890574
Precision (Macro): 0.6069117489399285 | SD: 0.0233000519041581
Recall (Micro): 0.7023524995199473 | SD: 0.04362806716890574
Recall (Macro): 0.6188593397583426 | SD: 0.01255799053482864
MLP Derived 2 Data (Custom Parameter)
Accuracy: 0.6651867922677686 | SD: 0.06649992329133163
Precision (Micro): 0.6651867922677686 | SD: 0.06649992329133163
Precision (Macro): 0.5798315491889792 | SD: 0.014593480797104897
Recall (Micro): 0.6651867922677686 | SD: 0.06649992329133163
Recall (Macro): 0.5847826994465

# Comparing default parameters with custom parameters

## Comparing learning rate default with learning rate custom

We changed the parameter "learning rate" to become an adaptive one instead of a constant. "Constant" is a constant rate given by "learning rate init" and "adaptive" keeps constant to "learning rate init" but only if training loss continues to decrease. It divides by 5 for every 2 consecutive fails to decrease training loss or fail to increase validation score.

The two results are below so we can compare:

Learning Rate = Constant
```
MLP Original Data
Accuracy: 0.7526297558017293 | SD: 0.05728441566919431
Precision (Micro): 0.7526297558017293 | SD: 0.05728441566919431
Precision (Macro): 0.6574139066691673 | SD: 0.011893024194706897
Recall (Micro): 0.7526297558017293 | SD: 0.05728441566919431
Recall (Macro): 0.6596520012090223 | SD: 0.0077860501918799105
MLP Derived 1 Data
Accuracy: 0.7045830173274815 | SD: 0.04228638563238359
Precision (Micro): 0.7045830173274815 | SD: 0.04228638563238359
Precision (Macro): 0.6072071696573753 | SD: 0.02209754869433878
Recall (Micro): 0.7045830173274815 | SD: 0.04228638563238359
Recall (Macro): 0.6195157883091523 | SD: 0.0086934773882508
MLP Derived 2 Data
Accuracy: 0.6582164975085615 | SD: 0.06432506626970061
Precision (Micro): 0.6582164975085615 | SD: 0.06432506626970061
Precision (Macro): 0.5739112344084412 | SD: 0.01598142091544596
Recall (Micro): 0.6582164975085615 | SD: 0.06432506626970061
Recall (Macro): 0.5807931570906334 | SD: 0.00941861958807479
```

Learning Rate = Adaptive
```
MLP Original Data (Custom Parameter)
Accuracy: 0.7530013303328318 | SD: 0.05515907375272302
Precision (Micro): 0.7530013303328318 | SD: 0.05515907375272302
Precision (Macro): 0.6580091451164207 | SD: 0.01369859669343959
Recall (Micro): 0.7530013303328318 | SD: 0.05515907375272302
Recall (Macro): 0.6608877356508466 | SD: 0.008622120813757373
MLP Derived 1 Data (Custom Parameter)
Accuracy: 0.7036537874423765 | SD: 0.04299811615073797
Precision (Micro): 0.7036537874423765 | SD: 0.04299811615073797
Precision (Macro): 0.6056796924886272 | SD: 0.021692283209374524
Recall (Micro): 0.7036537874423765 | SD: 0.04299811615073797
Recall (Macro): 0.6175541459084531 | SD: 0.008881357419671118
MLP Derived 2 Data (Custom Parameter)
Accuracy: 0.6537560144720319 | SD: 0.0658150325771587
Precision (Micro): 0.6537560144720319 | SD: 0.0658150325771587
Precision (Macro): 0.570238699382446 | SD: 0.014253218148431383
Recall (Micro): 0.6537560144720319 | SD: 0.0658150325771587
Recall (Macro): 0.5767813623524441 | SD: 0.012884803777157155
```

Between the two, it seems that Adaptive learning rate is slightly less accurate and offers lower precision and recall. I am assuming adaptive would be expecting much more training data in order to properly adapt. It also resulted in a higher standard deviation, which we should be able to lower with a larger dataset.



## Compare max iter default with custom max iter

We changed the parameter "max iter" to become half of the default. This will stop the convergence early once the iterations max has been reached. By cutting it in half, we expect it to be less accurate, as it allows less iterations to train with.

This change to half the max iterations took ~6 min per dataset for running the model.

The two results are below so we can compare:

Max Iterations = 200
```
MLP Original Data
Accuracy: 0.7526297558017293 | SD: 0.05728441566919431
Precision (Micro): 0.7526297558017293 | SD: 0.05728441566919431
Precision (Macro): 0.6574139066691673 | SD: 0.011893024194706897
Recall (Micro): 0.7526297558017293 | SD: 0.05728441566919431
Recall (Macro): 0.6596520012090223 | SD: 0.0077860501918799105
MLP Derived 1 Data
Accuracy: 0.7045830173274815 | SD: 0.04228638563238359
Precision (Micro): 0.7045830173274815 | SD: 0.04228638563238359
Precision (Macro): 0.6072071696573753 | SD: 0.02209754869433878
Recall (Micro): 0.7045830173274815 | SD: 0.04228638563238359
Recall (Macro): 0.6195157883091523 | SD: 0.0086934773882508
MLP Derived 2 Data
Accuracy: 0.6582164975085615 | SD: 0.06432506626970061
Precision (Micro): 0.6582164975085615 | SD: 0.06432506626970061
Precision (Macro): 0.5739112344084412 | SD: 0.01598142091544596
Recall (Micro): 0.6582164975085615 | SD: 0.06432506626970061
Recall (Macro): 0.5807931570906334 | SD: 0.00941861958807479
```

Max Iterations = 100
```
MLP Original Data (Custom Parameter)
Accuracy: 0.7517003877719894 | SD: 0.05479951527999845
Precision (Micro): 0.7517003877719894 | SD: 0.05479951527999845
Precision (Macro): 0.6563141348097064 | SD: 0.012381907763841111
Recall (Micro): 0.7517003877719894 | SD: 0.05479951527999845
Recall (Macro): 0.6589889671637601 | SD: 0.007750414878090436
MLP Derived 1 Data (Custom Parameter)
Accuracy: 0.7023524995199473 | SD: 0.04362806716890574
Precision (Micro): 0.7023524995199473 | SD: 0.04362806716890574
Precision (Macro): 0.6069117489399285 | SD: 0.0233000519041581
Recall (Micro): 0.7023524995199473 | SD: 0.04362806716890574
Recall (Macro): 0.6188593397583426 | SD: 0.01255799053482864
MLP Derived 2 Data (Custom Parameter)
Accuracy: 0.6651867922677686 | SD: 0.06649992329133163
Precision (Micro): 0.6651867922677686 | SD: 0.06649992329133163
Precision (Macro): 0.5798315491889792 | SD: 0.014593480797104897
Recall (Micro): 0.6651867922677686 | SD: 0.06649992329133163
Recall (Macro): 0.5847826994465168 | SD: 0.011236531504226682
```

Running only half the max iterations, the default 200 max iterations was slightly more accurate. This was due to the fact that it is harder to hit the 200 max iterations versus the 100 max iterations, resulting in higher quality training. Though not that big of a difference overall in micro and macro, I still strongly believe a larger dataset would have raised accuracy and the precision recall on the neural network.

# Table of all 12 model runs
| MLP Model         | Learn Rate | Max Iter | Accur.             | Accur. SD           | Prec. Micro        | Prec. Micro SD      | Prec. Macro        | Prec. Macro SD       | Recall Micro       | Rec Micro SD        | Recall Macro       | Rec Macro SD          |
|-------------------|------------|----------|--------------------|---------------------|--------------------|---------------------|--------------------|----------------------|--------------------|---------------------|--------------------|-----------------------|
| Original Default  | Constant   | 200      | 0.7526297558017293 | 0.05728441566919431 | 0.7526297558017293 | 0.05728441566919431 | 0.6574139066691673 | 0.011893024194706897 | 0.7526297558017293 | 0.05728441566919431 | 0.6596520012090223 | 0.0077860501918799105 |
| Deriv 1 Default   | Constant   | 200      | 0.7045830173274815 | 0.04228638563238359 | 0.7045830173274815 | 0.04228638563238359 | 0.6072071696573753 | 0.02209754869433878  | 0.7045830173274815 | 0.04228638563238359 | 0.6195157883091523 | 0.0086934773882508    |
| Deriv 2 Default   | Constant   | 200      | 0.6582164975085615 | 0.06432506626970061 | 0.6582164975085615 | 0.04228638563238359 | 0.6072071696573753 | 0.02209754869433878  | 0.7045830173274815 | 0.04228638563238359 | 0.6195157883091523 | 0.0086934773882508    |
| Original Adaptive | Adaptive   | 200      | 0.7530013303328318 | 0.05515907375272302 | 0.7530013303328318 | 0.05515907375272302 | 0.6580091451164207 | 0.01369859669343959  | 0.7530013303328318 | 0.05515907375272302 | 0.6608877356508466 | 0.008622120813757373  |
| Deriv 1 Adaptive  | Adaptive   | 200      | 0.7036537874423765 | 0.04299811615073797 | 0.7036537874423765 | 0.04299811615073797 | 0.6056796924886272 | 0.021692283209374524 | 0.7036537874423765 | 0.04299811615073797 | 0.6175541459084531 | 0.008881357419671118  |
| Deriv 2 Adaptive  | Adaptive   | 200      | 0.6537560144720319 | 0.0658150325771587  | 0.6537560144720319 | 0.0658150325771587  | 0.570238699382446  | 0.014253218148431383 | 0.6537560144720319 | 0.0658150325771587  | 0.5767813623524441 | 0.012884803777157155  |
| Original MI 100   | Constant   | 100      | 0.7517003877719894 | 0.05479951527999845 | 0.7517003877719894 | 0.05479951527999845 | 0.6563141348097064 | 0.012381907763841111 | 0.7517003877719894 | 0.05479951527999845 | 0.6589889671637601 | 0.007750414878090436  |
| Deriv 1 MI 100    | Constant   | 100      | 0.7023524995199473 | 0.04362806716890574 | 0.7023524995199473 | 0.04362806716890574 | 0.6069117489399285 | 0.0233000519041581   | 0.7023524995199473 | 0.04362806716890574 | 0.6188593397583426 | 0.01255799053482864   |
| Deriv 2 MI 100    | Constant   | 100      | 0.6651867922677686 | 0.06649992329133163 | 0.6651867922677686 | 0.06649992329133163 | 0.5798315491889792 | 0.014593480797104897 | 0.6651867922677686 | 0.06649992329133163 | 0.5847826994465168 | 0.011236531504226682  |

# Discussion of the 12 runs
---
Above is the comparison table of the 12 runs of the MLP model.

Notable things:
- Derived dataset 1 with Adaptive learning rate was the most accurate
- The original adaptive was the most precise on micro level
- The original adaptive was the most precise on a macro level
- The original adaptive had most recall on micro level
- The original default had the most recall on macro level

Overall, precision would be best on the original without any filtering tokens. In the other columns, original adaptive is moderately successful at analyzing and determining.

I would assume this is due to the adaptive learning rate that it has, which, despite having a reduced dataset to work with, which is then further reduced into a training and testing set, contributes to the current low scores.

The worst in terms of accuracy was derived dataset 2 with max iterations = 100. However, in general, the derived dataset 2 had a weak performance when being filtered, and that also goes for derived dataset 1. I believe that the derived datasets were weaker due to the lesser amounts of tokens that were being transformed and used for the training and testing.

However with larger datasets, it would be understandable to use derived datasets due to their reduced sizes compared to the unaltered original dataset. It would all be based on cost and time, more computing power and more time? Or add more datasets for more accuracy, and so on.


# References
### Scapy, Sklearn, Numpy, Pandas
- https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
- https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html
- https://ioflood.com/blog/train-test-split-sklearn/#:~:text=The%20train_test_split%20function%20is%20a%20powerful%20tool%20in%20Scikit%2Dlearn's,contains%20utilities%20for%20splitting%20data.
- https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#:~:text=The%20purpose%20of%20the%20pipeline,as%20in%20the%20example%20below.
- https://scikit-learn.org/stable/modules/cross_validation.html
- Creating a Logistic Regression and checking accuracy - https://www.datacamp.com/tutorial/understanding-logistic-regression-python
- make_scorer - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html
- Logistic Regression - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
- https://colab.research.google.com/drive/1KM-9ZwoxG2kdaCO1l-tFwUFea45Xzntn#scrollTo=XbJ_aos4YrXz (My Assignment 2)
- https://spacy.io/usage/linguistic-features
- https://spacy.io/api/doc
- https://universaldependencies.org/u/pos/
- https://www.kaggle.com/datasets/malharkhatu/airline-passenger-reviews
- https://github.com/baharin/CSI4106-Assignment4-Datasets/blob/main/reduced_file_AirPassengerReviews.csv
